In [ ]:
import os
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt

# Input and output folder paths
input_folder_path = r"File_Path_Input"
output_folder_path = r"File_Path_Output"

# Ensure the output folder exists, create it if not
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Loop through each file in the input folder
for file_name in os.listdir(input_folder_path):
    if file_name.endswith(".csv"):
        # Build the full file paths
        file_path = os.path.join(input_folder_path, file_name)
        output_file_path = os.path.join(output_folder_path, f"{os.path.splitext(file_name)[0]}_FORECAST.csv")

        # Load the CSV file into a DataFrame with a specified encoding
        data = pd.read_csv(file_path, encoding='ISO-8859-1')

        # Convert 'Date' to datetime type
        data['Date'] = pd.to_datetime(data['Date'])

        # List of columns to forecast
        forecast_columns = [
            'Minimum Temperature (°C)',
            'Maximum Temperature (°C)',
            'Minimum Relative Humidity (%)',
            'Maximum Relative Humidity (%)',
            'Rainfall (mm)',
            'Average 10m Wind Speed (m/sec)',
            'Evapo-Transpiration (mm)',
            'Solar Radiation (MJ/sq m)'
        ]

        # Convert numeric columns to numeric data type
        numeric_columns = [
            'Minimum Temperature (°C)',
            'Maximum Temperature (°C)',
            'Minimum Relative Humidity (%)',
            'Maximum Relative Humidity (%)',
            'Rainfall (mm)',
            'Average 10m Wind Speed (m/sec)',
            'Evapo-Transpiration (mm)',
            'Solar Radiation (MJ/sq m)'
        ]

        # Remove unwanted columns (Latitude, Longitude, Station Point, State)
        data = data[['Date', 'Station Name'] + forecast_columns]

        data[numeric_columns] = data[numeric_columns].apply(pd.to_numeric, errors='coerce')

        # Replace null values with the value from the row above
        data[numeric_columns] = data[numeric_columns].fillna(method='ffill')

        # Initialize an empty list to store DataFrames and Figures
        forecast_dfs = []

        # Loop through each column and create a separate forecast
        for column in forecast_columns:
            # Prepare the data in the required format for Prophet
            prophet_data = data[['Date', 'Station Name', column]].rename(columns={'Date': 'ds', 'Station Name': 'station', column: 'y'})

            # Initialize the Prophet model
            model = Prophet()

            # Fit the model with the data
            model.fit(prophet_data)

            # Create a DataFrame with future dates for prediction
            future = model.make_future_dataframe(periods=365 * 5)  # 5 years forecast

            # Generate the forecast
            forecast = model.predict(future)

            # Append the forecast DataFrame to the list
            forecast_dfs.append(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].rename(columns={'yhat': f'{column} Forecast', 'yhat_lower': f'{column} Lower', 'yhat_upper': f'{column} Upper'}))

        # Combine all forecasts into a single DataFrame
        forecast_combined = pd.concat([df.set_index('ds') for df in forecast_dfs], axis=1)

        # Reset the index to make 'ds' a regular column
        forecast_combined = forecast_combined.reset_index()

        # Merge the 'station' and the forecasted DataFrames on the index
        forecast_combined = pd.merge(data[['Date', 'Station Name']], forecast_combined, left_on='Date', right_on='ds', how='right')

        # Drop the redundant 'ds' column
        forecast_combined = forecast_combined.drop(columns=['ds'])

        # Save the combined forecast DataFrame to a CSV file in the Forecast folder
        forecast_combined.to_csv(output_file_path, index=False)

        print(f"Forecast saved for {file_name}")

print("Forecasting completed.")
